# 03 - feature engineering

Time for feature engineering. Because we're going to look at scaling inputs as part of the modelling process, we don't cover that here.

We're looking at the following flow:

<img src="../docs/imgs/energy-sa-feature-engineering.png " width="300">

So we're performing the following steps:
- Obtaining and processing the UK bank holidays data from the gov site, you'll learn to work with json content.
- Identifying from our data which region of the UK each LV feeder belongs to. We're going to obtain some geographical data that includes the relevant administrative boundaries for the UK, then join it based on if an LV feeder is within that region using DBSQLs spatial features.
- We introduce and join our weather data based on both time and nearest point to get weather features included.
- We look at normalizing our dependant variable based on the number of properties served by each LV feeder, then examine the impact of outliers and scale.
- We calculate some lagged values using window functions to use as regressors in our model.
- We introduce some basic frequency domain features by calculating the cyclical positions of different time periods. For example a weekly cycle can be thought of as 7 distinct steps, which can be mapped onto a unit circle with a `sin()` and `cos()` function to give the -1 to 1 scale feature for position-in-cycle.

Finally, we split our output into test, train and holdout. We holdout a small sample of LV feeders entirely for blind testing.

In [0]:
%run ./includes/common_functions_and_imports

In [0]:
source_table_name_energy = (
    f"{CONFIG.target_catalog}.{CONFIG.target_schema}.smart_meter_data_clean"
)
source_table_name_weather = (
  f"{CONFIG.target_catalog}.{CONFIG.target_schema}.weather_data_clean"
)

if not spark.catalog.tableExists(source_table_name_energy) or not spark.catalog.tableExists(source_table_name_weather):
  dbutils.notebook.exit('One of our sources does not exist')

clean_meter_df = spark.table(source_table_name_energy)
weather_pivoted_df = spark.table(source_table_name_weather)

In [0]:
target_table_name_administrative_boundaries_ref = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.gb_administrative_boundaries"
target_table_name_uk_bank_holidays = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.uk_bank_holidays"
target_table_name_unique_locations_with_admin_boundaries = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.energy_locations_with_admin_boundaries"

target_table_name_half_hourly_statistics = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.half_hourly_statistics_and_bounds"
target_table_name_ref_unique_feeders_with_holdout = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.ref_unique_feeders_with_holdout"

target_table_name_holdout_features = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.unscaled_holdout_features"
target_table_name_train = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.unscaled_train_features"
target_table_name_test = f"{CONFIG.target_catalog}.{CONFIG.target_schema}.unscaled_test_features"


## Public holidays and geographical processing

In order to make better forecasts, we can derive some very simple features to include in our model.

Public holidays would be a great example of one such feature, but the UK is, of course, a union of four different countries and therefore holidays differ by region 😬



### Part 1: The UK is multiple countries

Luckily there is a well-maintained public dataset of administrative boundaries hosted as part of the [Overture Maps](https://overturemaps.org/) project and we can use these to allocate each of our feeder locations to a particular country.

We can go ahead and read the 'division areas' data from Overture, directly from their S3 bucket. We'll use the new Databricks Spatial SQL functions to simplify these geometries a little and further speed up our imminent join.

In [0]:
overture_release = "2025-03-19.0"
divisions_theme = "divisions"
divisions_type = "division_area"

overture_divisions_uri = f"s3a://overturemaps-us-west-2/release/{overture_release}/theme={divisions_theme}/type={divisions_type}/"

#Only re-read if we're set to overwrite or the table doesn't exist.
if not spark.catalog.tableExists(target_table_name_administrative_boundaries_ref) or CONFIG.overwrite_data:
  (
      spark.read.parquet(overture_divisions_uri)
      .where(F.col("subtype") == "region")
      .where("country = 'GB'")
      .selectExpr(
          "st_aswkb(st_simplify(st_geomfromwkb(geometry), 0.001)) as region_geom",
          "region as region_name",
      )
      .write.saveAsTable(
          target_table_name_administrative_boundaries_ref, mode="overwrite"
      )
  )

divisions_df = spark.table(target_table_name_administrative_boundaries_ref)

For our next trick, we're going to save an intermediate result, which is the unique meter locations with the administrative boundaries.

We do this because the geometry join (st_contains) is not currently supported by photon, and later on we're going to leverage photon for our time series preparation.

The result is approximately 50K unique locations (x, y coordinates) and their respective admin boundaries.

In [0]:
join_expression_sql = F.expr(
    """
st_contains(
  st_geomfromwkb(region_geom), 
  st_point(locations.geometry.x, locations.geometry.y)
)
"""
)
if (not spark.catalog.tableExists(target_table_name_unique_locations_with_admin_boundaries)) or CONFIG.overwrite_data:
  (
      clean_meter_df.select('geometry').distinct().alias("locations")
      .join(divisions_df, on=join_expression_sql, how="inner")
      .drop("region_geom")
      .write.saveAsTable(
          target_table_name_unique_locations_with_admin_boundaries, mode="overwrite")
  )

energy_locations_with_admin_boundaries = spark.table(target_table_name_unique_locations_with_admin_boundaries)

### Part 2: Add in the holidays and weekends

The UK website conveniently provides all the bank holidays in json format, here: [https://www.gov.uk/bank-holidays.json](https://www.gov.uk/bank-holidays.json)

We're going to do the following:
- use pandas `read_json` to grab this straight from the web.
- This produces a dataframe with columns for each region and 2 rows:
  - row 1 is a duplicated region name (called 'division'), which we wil drop.
  - row 2 which contains the json content of bank holidays for that region.
- We melt this frame to pivot it into two columns:
  - region
  - Array of structs (our json content)
- Now we have 2 columns x 3 rows for each region in the UK.
- We turn this into a spark frame, then use the magical [inline](https://docs.databricks.com/aws/en/sql/language-manual/functions/inline) command to automatically explode and expand our json content out into 1 row per holiday x N columns of struct data.

**The result is:**
7 columns by 252 rows of holidays.

Finally just for good measure, we'll add a column indicating if this is a weekend as we'd expect a deviation in domestic consumption patterns.

In [0]:
if not spark.catalog.tableExists(target_table_name_uk_bank_holidays) or CONFIG.overwrite_data:

  pdf = pd.read_json('https://www.gov.uk/bank-holidays.json')
  bank_holidays_df = spark.createDataFrame(pdf.drop('division').melt()).withColumnsRenamed({'variable': 'division', 'value':'holidays_array'})

  lookup_location_df = spark.createDataFrame(
    [
      ('GB-ENG', 'england-and-wales'),
      ('GB-NIR', 'northern-ireland'),
      ('GB-SCT', 'scotland'),
      ('GB-WLS', 'wales')
      ], 'region_name string, division string'
    )

  bank_holidays_df = bank_holidays_df.join(lookup_location_df, on='division', how='left')
  # We're going to use inline here to seamlessly process this json from many-rows-and-columns in a single cell per row into a normal table.
  bank_holidays_df.selectExpr('division','region_name','inline(holidays_array)', 'True as is_bank_holiday').write.saveAsTable(target_table_name_uk_bank_holidays, mode='overwrite')

bank_holidays_df = spark.table(target_table_name_uk_bank_holidays)

In [0]:
energy_data_with_features = (
    clean_meter_df.withColumn(
        "date", F.to_date("data_collection_log_timestamp")
    )
    .alias("meter")
    .join(energy_locations_with_admin_boundaries, on="geometry", how='inner')
    .alias('meter_with_admin_boundaries')
    .join(
        bank_holidays_df.select(
            "date", "region_name", "is_bank_holiday", "title"
        ).alias("hols"),
        on=["date", "region_name"],
        how="left",
    )
    .withColumn(
        "half_hour",
        F.hour("data_collection_log_timestamp") * 60
        + F.minute("data_collection_log_timestamp"),
    ).withColumn('hour', F.hour("data_collection_log_timestamp"))
    .withColumn("is_bank_holiday", F.col("is_bank_holiday").isNotNull())
    .withColumn(
        "is_weekend", F.weekday("data_collection_log_timestamp") >= 5
    )  # 0 indexed on monday, so 5 = saturday
    .withColumn('weekday_monday_eq_0', F.weekday("data_collection_log_timestamp"))
)

### Part 3: add in weather data based on geography and nearest time interval

So we have some energy meter data and some weather historicals. Now we need to join on two quite complex things: Geography (nearest weather measurement point to LV feeder), and time period (half hourly meter meets hourly weather).

Luckily, we made a simplification earlier! We collected our weather historicals at 0.25 deg intervals. We can use this to do a lazy-neighbour approach for our LV feeders to massively simplify our geo-problem, so we'll round our LV feeder locations to the nearest 0.25 degrees.

For our time alignment we can use the same approach and round to the nearest hour. We could also use a window function to interpolate the weather values to half hourly periods, but this would be another chunk of processing and would double the size of our weather dataset.

In [0]:
energy_data_with_weather_join = (
    energy_data_with_features
      .withColumn("rounded_x", F.round(F.col("geometry.x") / 0.25) * 0.25)
      .withColumn("rounded_y", F.round(F.col("geometry.y") / 0.25) * 0.25)
      .withColumn("rounded_timestamp",
                  F.date_trunc("hour", F.col("data_collection_log_timestamp"))
                 )
    .alias("energy")
    .join(
      weather_pivoted_df.alias("locations"),
      on=[
          F.col("energy.rounded_x") == F.col("locations.x"),
          F.col("energy.rounded_y") == F.col("locations.y"),
          F.col("energy.rounded_timestamp") == F.col("locations.time"),
      ],
      how="left",
    )
    # Calculate the euclidean distance between coordinates, may be useful to compare against prediction errors and weather.
    .withColumn('approx_distance_to_weather_station', F.sqrt((F.col('x')-F.col('geometry.x'))**2 + (F.col('y')-F.col('geometry.y'))**2))
    # Tidy up column order and names : keys, dependant variable, features...
    .select(
        "lv_feeder_unique_id",
        "secondary_substation_unique_id",
        "dno_alias",
        "data_collection_log_timestamp",
        "hour",
        "half_hour",
        F.col("time").alias("weather_forecast_time"),
        F.col("geometry").alias("lv_feeder_geometry"),
        F.struct("x", "y").alias("weather_forecast_geometry"),
        "approx_distance_to_weather_station",
        "aggregated_device_count_active",
        "total_consumption_active_import",
        "is_weekend",
        "weekday_monday_eq_0",
        "is_bank_holiday",
        F.col("title").alias("bank_holiday_name"),
        "t2m",
        "u10",
        "v10",
        "ssrd",
        "strd"
    )
)

Databricks visualization. Run in Databricks to view.

Watch out for: rows with no available weather data. It's possible depending on your dataset that you would end up with a bunch of a rows that you have no weather data to join to, which you would need to decide how to handle. A cheap and lazy method would be to inner join the two and cut off any results lacking data, but you may miss a section of your time data you're interested in (maybe you need to use forecast weather for the latest smart meter readings?)

## Preparing our time series

### Basic dependant variable processing

The very first thing we need to do wih our consumption figures is normalise them against the number of active devices for each low voltage feeder. Currently they're all on different scales depending how many devices are served.

From our data exploration we know the minimum value of `aggregated_device_count_active` is 5, so we don't anticipate any infinite responses.

There is two plots below showing the difference pre- and post- normalization. It should be pretty apparent that pre-normalization would lead you to believe that every LV_feeder has a very different profile.



In [0]:
display(spark.sql(
  f"""
with first_n_feeders as (
  select distinct
    lv_feeder_unique_id
  from
    {source_table_name_energy}
  limit 10
)
select
  md.lv_feeder_unique_id,
  date_trunc('DAY', data_collection_log_timestamp) as date,
  sum(total_consumption_active_import/1000) as daily_consumption,
  sum(total_consumption_active_import/1000)
  / first(aggregated_device_count_active) as daily_consumption_normalized
from
  {source_table_name_energy} md
    inner join first_n_feeders
      on md.lv_feeder_unique_id = first_n_feeders.lv_feeder_unique_id
group by
  md.lv_feeder_unique_id,
  date
"""
))

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
normalized_df = energy_data_with_weather_join.withColumn(
    "normalized_consumption_kwh",
    F.try_divide(
        F.col("total_consumption_active_import"),
        F.col("aggregated_device_count_active"),
    )/1000,
)

If we had access to more information about LV feeders (such as model, installed capacity, max properties served, etc), we could look at performing some more advanced analytics. An LV feeder is a good independant variable - The details would be readily accessible to a supplier, so we will always have access to it. If it turns out that different classes of feeder have different characteristics (for example, we cannot have EV charger consumption on model xyz), then this becomes a meaningful predictor for consumption.

### Adding in lagged values

Lagged values are useful for regression models, and since we're dealing with half hourly periods may act as useful features for identifying patterns ()

In [0]:
my_window = Window.partitionBy("lv_feeder_unique_id").orderBy(
    "data_collection_log_timestamp"
)
consumption_column = "normalized_consumption_kwh"
lagged_df = normalized_df.withColumn(
    "lagged_values",
    F.struct(
        (
            F.lag(
                consumption_column, 1, default=F.col(consumption_column)
            ).over(my_window)
        ).alias("lag_1"),
        (
            F.lag(
                consumption_column, 2, default=F.col(consumption_column)
            ).over(my_window)
        ).alias("lag_2"),
        (
            F.lag(
                consumption_column, 3, default=F.col(consumption_column)
            ).over(my_window)
        ).alias("lag_3"),
    ),
).withColumn(
    "diffed_timestamp",
        (
            F.col("data_collection_log_timestamp")
            - F.lag(
                "data_collection_log_timestamp",
                1,
                default=F.col("data_collection_log_timestamp"),
            ).over(my_window)
        ).cast('int')
        
)

Databricks visualization. Run in Databricks to view.

### Adding some cyclical indicators

| cyclical component | periods |
|-|-|
|half-hour | 48 |
|hour| 24|
|day|7|
|week|52|
|month|12|

In [0]:
def add_cyclical_features(df, col_name, period):
    angle = F.col(col_name) * (2 * F.pi() / period)
    return df.withColumn(f"{col_name}_sin", F.sin(angle)).withColumn(
        f"{col_name}_cos", F.cos(angle)
    )

# Standardise all column names for cyclicals, and set them all to start at zero then run to their respective periods:

lagged_df_with_cyclicals = (
    lagged_df.withColumn("cyc_halfhour", F.col("half_hour") / 30)
    .withColumnRenamed("hour", "cyc_hour")
    .withColumnRenamed("weekday_monday_eq_0", "cyc_day")
    .withColumn("cyc_week", F.weekofyear("data_collection_log_timestamp") - 1)
    .withColumn("cyc_month", F.month("data_collection_log_timestamp") - 1)
)
cyclical_features = {"cyc_halfhour":48, "cyc_hour":24, "cyc_day":7, "cyc_week":52, "cyc_month":12}

for col_name, period in cyclical_features.items():
  lagged_df_with_cyclicals = add_cyclical_features(lagged_df_with_cyclicals, col_name, period)

In [0]:
display(lagged_df_with_cyclicals)

Databricks visualization. Run in Databricks to view.

### Thinking about outliers (again)

we've previously removed outliers very loosely with an aim at keeping statistically significant data in.

We've picked up on a few more - Notably:
- 3 LV feeders with values surpassing 35kWh in a HH period after being normalized for device counts.
- periods with unexplainable anomalous behaviour - For example filtering the normalized usage to >10K will show you LV feeders on a scatter plot which appear to just linearly increase continually for a number of weeks on end. We've included one example below.

While we suspect these may be erroneous readings, we have no way of easily isolating them, or investigating further at a device level.

Fun activity: Zoom into the trending increases, and you get what appear to be valid consumption patterns.

In [0]:
display(normalized_df.filter("lv_feeder_unique_id = 'SSEN-372100150001'"))

Databricks visualization. Run in Databricks to view.

We also have a situation where we know removal methods like the zscore would be unfairly influenced by outliers, and we can't assume that the IQR would work.

 If we plotted a histogram of the log-scale consumption (below) we'd see it's comically left skewed towards the 'hundreds' order of magnitude (2) - This makes sense, we're dealing with electrical consumption, and most of the day people arn't running multi-kW draw devices.

 We can see however, if we look at the change in log scale over time represented as a percentage, how usage shifts and peaks around 6-6:30pm, and the fraction of responses at magnitude 'thousands' (or kW draw) is much higher.

 


In [0]:
display(
    normalized_df.dropna(how="any", subset=[consumption_column])
    .withColumn("Log_scale_import", F.round(F.log10(consumption_column)))
    .groupBy((F.col('half_hour')/60).alias('half_hour'),'Log_scale_import').agg(F.count('*').alias('count'))
)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

So we're going to take a slightly different approach.

We're going to create a reference table with the IQR limits calculated for **each half hour**, based on both:
- normalized consumption value, and,
- log10 normalized consumption value

This will give us bounds we can join into our datasets and use if we like. We also get some useful statistics like 'how many data points do we have for each half-an-hour'.

In [0]:
hh_statistics = (
    lagged_df_with_cyclicals.groupBy('cyc_halfhour')
    .agg(
        # bounds
        F.percentile(consumption_column, 0.25).alias("p25"),
        F.percentile(consumption_column, 0.75).alias("p75"),
        (
            F.percentile(consumption_column, 0.75)
            - F.percentile(consumption_column, 0.25)
        ).alias("iqr"),
        (F.col("p25") - 1.5 * F.col("iqr")).alias("lower_bound"),
        (F.col("p75") + 1.5 * F.col("iqr")).alias("upper_bound"),
        # stats
        F.mean(consumption_column).alias("mean"),
        F.stddev(consumption_column).alias("stddev"),
        F.min(consumption_column).alias("min"),
        F.max(consumption_column).alias("max"),
        F.median(consumption_column).alias("median"),
        F.count(consumption_column).alias("nsamples"),
        # logscale outliers
        F.percentile(F.log10(consumption_column), 0.25).alias("log10_p25"),
        F.percentile(F.log10(consumption_column), 0.75).alias("log10_p75"),
        (
            F.percentile(F.log10(consumption_column), 0.75)
            - F.percentile(F.log10(consumption_column), 0.25)
        ).alias("log10_iqr"),
        (F.col("log10_p25") - 1.5 * F.col("log10_iqr")).alias("log10_lower_bound"),
        (F.col("log10_p75") + 1.5 * F.col("log10_iqr")).alias("log10_upper_bound"),
    )
    .orderBy("cyc_halfhour")
    .select(
        "cyc_halfhour",
        F.struct("p25", "p75", "iqr", "lower_bound", "upper_bound").alias(
            "hh_bounds"
        ),
        F.struct("mean", "stddev", "min", "max", "median", "nsamples").alias(
            "hh_stats"
        ),
        F.struct("log10_p25", "log10_p75", "log10_iqr", "log10_lower_bound", "log10_upper_bound").alias(
            'log10_bounds'
    )
))
if not spark.catalog.tableExists(target_table_name_half_hourly_statistics) or CONFIG.overwrite_data:
    (hh_statistics.write.option('overwriteSchema',True).saveAsTable(target_table_name_half_hourly_statistics, mode='overwrite'))

hh_statistics = spark.table(target_table_name_half_hourly_statistics)

Databricks visualization. Run in Databricks to view.

The final feature we're adding then is an indicator of if this is a suspected outlier. We'll keep outliers in the saved data and leave it to user discretion.

In [0]:
featured_engineered_all_data = lagged_df_with_cyclicals.join(
    hh_statistics.select("cyc_halfhour", "log10_bounds"), on="cyc_halfhour", how="inner"
).withColumn(
    "in_log10_iqr_limit",
    (F.col("normalized_consumption_kwh") >= 10 ** F.col("log10_bounds.log10_lower_bound"))
    & (
        F.col("normalized_consumption_kwh")
        <= 10 ** F.col("log10_bounds.log10_upper_bound")
    ),
)

## Splitting our data and saving the results

We're going to do a couple things here:
1. Reserve a fraction of LV feeder data traces entirely as a hold out validation.
2. Split the remaining data into a standard split based on time.

In [0]:
unique_lv_feeders = featured_engineered_all_data.select('lv_feeder_unique_id').distinct()
holdout_lv_feeders, remaining_data = unique_lv_feeders.randomSplit([0.05, 0.95], seed=42)

holdout_lv_feeders = holdout_lv_feeders.withColumn('holdout_feeder', F.lit(True))
remaining_data = remaining_data.withColumn('holdout_feeder', F.lit(False))

# save our table down for faster processing and use later.
if not spark.catalog.tableExists(target_table_name_ref_unique_feeders_with_holdout) or CONFIG.overwrite_data:
  remaining_data.unionByName(holdout_lv_feeders).write.saveAsTable(target_table_name_ref_unique_feeders_with_holdout, mode='overwrite')
  
holdout_feeder_reference = spark.table(target_table_name_ref_unique_feeders_with_holdout)

# This gives us around 6k holdout feeders and 116k included feeders.


Databricks visualization. Run in Databricks to view.

In [0]:
holdout_feature_data = featured_engineered_all_data.join(
    holdout_feeder_reference.filter("holdout_feeder"),
    on="lv_feeder_unique_id", how='inner')

remaining_feature_data = featured_engineered_all_data.join(
    holdout_feeder_reference.filter("not holdout_feeder"),
    on="lv_feeder_unique_id",
    how="inner",
)

# Training data: from 2024-01-01 to 2024-12-31
train_data = remaining_feature_data.orderBy("data_collection_log_timestamp").filter(
    "data_collection_log_timestamp between '2024-01-01' and '2024-12-31'"
)
# Testing data: from 2025-01-01 to 2025-03-29
test_data = remaining_feature_data.orderBy("data_collection_log_timestamp").filter(
    "data_collection_log_timestamp between '2025-01-01' and '2025-03-29'"
)

datasets_to_write = {
  target_table_name_holdout_features: holdout_feature_data,
  target_table_name_train: train_data,
  target_table_name_test: test_data
}

for dataset_name, dataset in datasets_to_write.items():
  if not spark.catalog.tableExists(dataset_name) or CONFIG.overwrite_data:
    print(f"writing: {dataset_name}")
    dataset.write.option('overwriteSchema', True).saveAsTable(dataset_name, mode='overwrite')